In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from pathlib import Path
from datetime import datetime as dt
import time, os
# import tensorflow.keras
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pydotplus
from IPython.display import Image


In [2]:
## function to genearte Confusion Matrix and Classification Report
def gen_cm_cr(modelName, y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(
        cm, index=["Legitimate 0", "Fraudulent 1"], columns=["Predicted Legitimate 0", "Predicted Fraudulent 1"]
    )
    
    # Calculating the accuracy score
    acc_score = accuracy_score(y_test, y_pred)
    print("Confusion Matrix: " + modelName)
    display(cm_df)
    print(f"Accuracy Score : {acc_score}")
    print("Classification Report")
    print(classification_report(y_test, y_pred))

In [3]:
## function to find the distance in kilometers between two geo-spatial co-ordinates.  Based on excample from Scikit Learn documentation
def get_distance(from_lat, from_lng, to_lat, to_lng):
    from sklearn.metrics.pairwise import haversine_distances
    from math import radians
    start =[]
    end =[]
    start.append(from_lat)
    start.append(to_lat)
    end.append(to_lat)
    end.append(to_lng)
    start_in_radians = [radians(_) for _ in start]
    end_in_radians = [radians(_) for _ in end]
    result = haversine_distances([start_in_radians, end_in_radians])
    result * 6371000/1000 
    return result
    

In [4]:
## function to assign the U.S. Bureau of Economic Analysis region to a state.
def get_region(state):
    
    states_to_bea_regions = {
        "AL": "Southeast",
        "AK": "Far West",
        "AZ": "Southwest",
        "AR": "Southeast",
        "CA": "Far West",
        "CO": "Rocky Mountain",
        "CT": "New England",
        "DE": "Mideast",
        "DC": "Mideast",
        "FL": "Southeast",
        "GA": "Southeast",
        "HI": "Far West",
        "ID": "Rocky Mountain",
        "IL": "Great Lakes",
        "IN": "Great Lakes",
        "IA": "Plains",
        "KS": "Plains",
        "KY": "Southeast",
        "LA": "Southeast",
        "ME": "New England",
        "MD": "Mideast",
        "MA": "New England",
        "MI": "Great Lakes",
        "MN": "Great Lakes",
        "MS": "Southeast",
        "MO": "Great Lakes",
        "MT": "Rocky Mountain",
        "NE": "Plains",
        "NV": "Southwest",
        "NH": "New England",
        "NJ": "Mideast",
        "NM": "Southwest",
        "NY": "Mideast",
        "NC": "Southeast",
        "ND": "Plains",
        "OH": "Great Lakes",
        "OK": "Southwest",
        "OR": "Far West",
        "PA": "Mideast",
        "RI": "New England",
        "SC": "Southeast",
        "SD": "Plains",
        "TN": "Southeast",
        "TX": "Southwest",
        "UT": "Rocky Mountain",
        "VT": "New England",
        "VA": "Mideast",
        "WA": "Far West",
        "WV": "Southeast",
        "WI": "Great Lakes",
        "WY": "Rocky Mountain"
    }
    region = states_to_bea_regions.get(state)
    return region



This dataset offers a variety of attributes valuable for comprehensive analysis. It contains 555,719 instances and 22 attributes, a mix of categorical and numerical data types. Importantly, the dataset is complete with no null values. Here's a breakdown of the attributes:  

1.t
Trans_date_trans_time: Timestamp of the transacti.  
2. cc
C  u m:Unique customer identificationum  r.  
3. merchant: antThe merchant involved in the transi     
4. c.
t::eg: ory Transaction type (e.g., perso, childcare).  .5.
a Amnt: Transaction amo.  
6. f.
First: Cardho first name.  
7. le.
: 
Last Cader dhst name.  
8. gme.
Gendrdholder gender.  
9. sder.
Street: Cer sardht address.  
10. cres: lCardholder city.  err  sdence.
State:er stater  
12. zip: Cardholder zip code   
13. lat: Latitude of cardholder.  
14. long: Longitude of cardholder.  
15. coca  tio Population of cardholder city.  
16. j.
r'job title of card holder.  
17. dob: Cardholder  
18.
18. 's d  
18. trans_num: unique transaction identifier.  
19. unix_time: transaction timestamp in unix format.  
20. merch_lat: latitude of merchant location.  
21. merch_long: longitude of merchant location.  
22.    (longi tude).
IsfraunsacFraudulent* tra stionndicator (1 = f**raud, **0 = legitie mate). This is thtargvariable for c***lassif**  ication purposes.

In [32]:
fraud_test_df = pd.read_csv('../Project-4 Team 6/Resources/fraud test.csv')

In [33]:
fraud_test_df.describe(include='all')

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,555719,5.557190e+05,555719,555719,555719.000000,555719,555719,555719,555719,...,555719.000000,555719.000000,5.557190e+05,555719,555719,555719,5.557190e+05,555719.000000,555719.000000,555719.000000
unique,NaN,226976,NaN,693,14,NaN,341,471,2,924,...,NaN,NaN,NaN,478,910,555719,NaN,NaN,NaN,NaN
top,NaN,15/12/2020 21:26,NaN,fraud_Kilback LLC,gas_transport,NaN,Christopher,Smith,F,444 Robert Mews,...,NaN,NaN,NaN,Film/video editor,23/03/1977,2da90c7d74bd46a0caf3777415b3ebd3,NaN,NaN,NaN,NaN
freq,NaN,16,NaN,1859,56370,NaN,11443,12146,304886,1474,...,NaN,NaN,NaN,4119,2408,1,NaN,NaN,NaN,NaN
mean,277859.000000,NaN,4.178387e+17,NaN,NaN,69.392810,NaN,NaN,NaN,NaN,...,38.543253,-90.231325,8.822189e+04,NaN,NaN,NaN,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,NaN,1.309837e+18,NaN,NaN,156.745941,NaN,NaN,NaN,NaN,...,5.061336,13.721780,3.003909e+05,NaN,NaN,NaN,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,NaN,6.041621e+10,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,...,20.027100,-165.672300,2.300000e+01,NaN,NaN,NaN,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,NaN,1.800430e+14,NaN,NaN,9.630000,NaN,NaN,NaN,NaN,...,34.668900,-96.798000,7.410000e+02,NaN,NaN,NaN,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,NaN,3.521420e+15,NaN,NaN,47.290000,NaN,NaN,NaN,NaN,...,39.371600,-87.476900,2.408000e+03,NaN,NaN,NaN,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,NaN,4.635330e+15,NaN,NaN,83.010000,NaN,NaN,NaN,NaN,...,41.894800,-80.175200,1.968500e+04,NaN,NaN,NaN,1.385867e+09,41.954163,-80.264637,0.000000


In [34]:
fraud_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  float64
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

# Preprocessing and Feature Engineering

In [35]:
# moving the Unamed column to the index and renaming it 'ID'
fraud_test_df.rename(columns={'Unnamed: 0':'ID'}, inplace=True)
fraud_test_df.set_index('ID', inplace=True, drop=True, verify_integrity ='True')

In [36]:
#creating cardholder age feature
fraud_test_df['date_dob'] = pd.to_datetime(fraud_test_df['dob'], format='%d/%m/%Y')
fraud_test_df['dt_trans_date_time'] = pd.to_datetime(fraud_test_df['trans_date_trans_time'], format='%d/%m/%Y %H:%M')
fraud_test_df['age_years'] = (dt.today()- fraud_test_df['date_dob'])/pd.Timedelta('365 days')
fraud_test_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,date_dob,dt_trans_date_time,age_years
ID,,,,,,,,,,,,,,,,,,,,,
0,21/06/2020 12:14,2.291160e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,Mechanical engineer,19/03/1968,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0,1968-03-19,2020-06-21 12:14:00,56.131322
1,21/06/2020 12:14,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,"Sales professional, IT",17/01/1990,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0,1990-01-17,2020-06-21 12:14:00,34.284747
2,21/06/2020 12:14,3.598220e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,"Librarian, public",21/10/1970,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0,1970-10-21,2020-06-21 12:14:00,53.539541
3,21/06/2020 12:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,Set designer,25/07/1987,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0,1987-07-25,2020-06-21 12:15:00,36.769678
4,21/06/2020 12:15,3.526830e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,Furniture designer,06/07/1955,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0,1955-07-06,2020-06-21 12:15:00,68.843651


In [37]:
fraud_test_df['merchant'].value_counts()

merchant
fraud_Kilback LLC                        1859
fraud_Cormier LLC                        1597
fraud_Schumm PLC                         1561
fraud_Kuhn LLC                           1521
fraud_Dickinson Ltd                      1519
                                         ... 
fraud_Treutel-King                        323
fraud_Satterfield-Lowe                    319
fraud_Kessler Group                       318
fraud_Jerde-Hermann                       312
fraud_Ritchie, Bradtke and Stiedemann     304
Name: count, Length: 693, dtype: int64

In [38]:
#creating distance between cardholder and merchant feature and economic region feature from the state abbreviation.
for index, row in fraud_test_df.iterrows():
    fraud_test_df.at[index, 'distance_km'] = get_distance(row['lat'], row['long'], row['merch_lat'], row['merch_long'])[0][1]
    fraud_test_df.at[index, 'region'] = get_region(row['state'])

In [39]:
# collapsing job categories to 'other' category for any category with fewer than 300 entries
job_counts=fraud_test_df['job'].value_counts()
job_counts_df = job_counts.to_frame(name='counts')
for index, row in job_counts_df.iterrows():
    if job_counts_df.at[index, 'counts'] < 300:
        fraud_test_df.loc[fraud_test_df['job'] == index, ['job']] = 'other'
        

In [40]:
fraud_test_df['job'].value_counts()

job
other                                                          12796
Film/video editor                                               4119
Exhibition designer                                             3968
Surveyor, land/geomatics                                        3756
Naval architect                                                 3750
                                                               ...  
Scientist, clinical (histocompatibility and immunogenetics)      413
Sports administrator                                             412
Ambulance person                                                 407
Plant breeder/geneticist                                         406
Media planner                                                    402
Name: count, Length: 418, dtype: int64

In [41]:
fraud_test_df['merchant'].value_counts()

merchant
fraud_Kilback LLC                        1859
fraud_Cormier LLC                        1597
fraud_Schumm PLC                         1561
fraud_Kuhn LLC                           1521
fraud_Dickinson Ltd                      1519
                                         ... 
fraud_Treutel-King                        323
fraud_Satterfield-Lowe                    319
fraud_Kessler Group                       318
fraud_Jerde-Hermann                       312
fraud_Ritchie, Bradtke and Stiedemann     304
Name: count, Length: 693, dtype: int64

In [42]:
fraud_test_df.to_csv('/Users/guranslimbu/Desktop/\\fraud_test_extended.csv', sep =',')

In [43]:
fraud_test_df.dtypes.head()

trans_date_trans_time     object
cc_num                   float64
merchant                  object
category                  object
amt                      float64
dtype: object

## Splitting Training and Testing data

## Feature Encoding
1. get_dummies/one_hot_encoders: Category, Gender, region
3. Target_encoder:  city, state, jobs, merchant, cc_num  

In [44]:
# Selecting features (X) and target variable (y)
y = fraud_test_df['is_fraud']  # Target variable
X = fraud_test_df[['amt', 'cc_num', 'lat', 'long', 'merch_lat', 'merch_long', 'age_years', 'distance_km', 'city_pop']]

In [48]:
# Selecting categorical columns for one-hot encoding
columns = ['merchant', 'category', 'gender', 'job', 'region']

# Create dummy variables for categorical columns
dummies_df = pd.get_dummies(fraud_test_df[columns])

In [49]:
# Concatenate the dummy variables with the original DataFrame
fraud_test_df = pd.concat([fraud_test_df, dummies_df], axis=1)

# Drop the original categorical columns
fraud_test_df.drop(categorical_columns, axis=1, inplace=True)

In [50]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# Display the shape of training and testing sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (444575, 9)
Shape of X_test: (111144, 9)
Shape of y_train: (444575,)
Shape of y_test: (111144,)


In [52]:
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [53]:
# Initialize and train the KNN model with k = 5 neighbours
model = KNeighborsClassifier(n_neighbors=5)

In [54]:
# Train & fit the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [55]:
# Create predictions
y_pred = model.predict(X_test_scaled)

In [64]:
# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[110663     55]
 [   248    178]]


In [67]:
# Classififcation Report
gen_cm_cr('classification_report', y_test, y_pred)

Confusion Matrix: classification_report


,Predicted Legitimate 0,Predicted Fraudulent 1
Legitimate 0,110663,55
Fraudulent 1,248,178


Accuracy Score : 0.9972738069531418
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    110718
           1       0.76      0.42      0.54       426

    accuracy                           1.00    111144
   macro avg       0.88      0.71      0.77    111144
weighted avg       1.00      1.00      1.00    111144



In [69]:
# Evaluate the classifier's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9972738069531418


The model demonstrates high accuracy for legitimate transactions, however it may need improvement in detecting fraudulent transactions, as indicated by the lower precision, recall, and F1-score for class 1. Further analysis of Random Forest testing on training model is performed to enhance the model's ability to detect fraud.

# Random Forest Testing

In [76]:
# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)  

In [ ]:
# Train & fit the model
rf_model.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate Model
y_pred = rf_model.predict(X_train_scaled)

In [ ]:
# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Classification Report
gen_cm_cr('classification_report', y_test, y_pred)

In [ ]:
# Print accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)